In [1]:
import matplotlib
matplotlib.use("svg")
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("ggplot")
from mpl_toolkits.axes_grid1 import ImageGrid
from scipy import spatial
from pandas import DataFrame as df
from datetime import datetime
import pandas as pd
import numpy as np
import collaborative_filtering as cf
import linear_regression as lr
import factorization_machine as fm
import networkx as nx
import seaborn as sns
from pyfm import pylibfm
import movie_parsing
import process_data
from sklearn.feature_extraction import DictVectorizer

links = pd.read_csv('./data/ml-latest-small/links.csv')
movies = pd.read_csv("./data/ml-latest-small/movies.csv")
ratings = pd.read_csv("./data/ml-latest-small/ratings.csv")
df_movies = pd.read_csv('./data/movie_objects.csv', encoding = 'utf-8')
df_user = pd.read_csv("./data/u.user",sep='|', 
                      names=['userId', 'age', 'gender', 'occupation', 'zip_code'], encoding = 'utf-8')[:671]

reload(process_data)
df_movies = movie_parsing.parse(df_movies)
# df_movie_features, df_user_features = process_data.process(df_movies, df_user)
# df_movie_features, df_user_features = df_movie_features.set_index('imdbId'), df_user_features.set_index('userId')
ratings = ratings.merge(links, on = 'movieId')
ratings = ratings.merge(df_movies, on ='imdbId')
ratings = ratings.merge(df_user, on = 'userId')
ratings.dropna(subset=['year'],inplace=True)

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
from sklearn.feature_extraction import DictVectorizer
from pyfm import pylibfm
from scipy.sparse import csr_matrix, vstack, hstack, csc_matrix
import math

np.random.seed(10)
P = np.random.permutation(len(ratings))
n = len(ratings)
ratings_tr = ratings.iloc[P[:int(math.floor(9*n /10.))]]
ratings_te = ratings.iloc[P[int(math.floor(9*n /10.)):]]

def data_generate(row):
    movie_feature = ['imdbId_onehot']
    user_feature = ['userId_onehot']
    
    u = df_user_features.loc[int(row[1]['userId']), user_feature][0]
    for uu in df_user_features.loc[int(row[1]['userId']), user_feature][1:]:
        u = hstack(u, uu, format = 'csc')
    v = df_movie_features.loc[int(row[1]['imdbId']), movie_feature][0]
    for vv in df_movie_features.loc[int(row[1]['imdbId']), movie_feature][1:]:
        v = hstack(v, vv, format = 'csc')
        
    return hstack((u, v))
        
    
ratings_tr['data'] = map(data_generate, ratings_tr.iterrows())

In [ ]:
np.random.seed(10)
# Collaberative Filtering
reload(cf)
X_tr, X_te, movieNames, idxToUserId, idxToLenId = cf.process(ratings, movies, np.random.permutation(len(ratings)))
U_cf, V_cf = cf.train(X_tr, X_te, k = 3, lam = 1, niters = 0, verbose = True)

In [ ]:
# Linear Regression
reload(lr)
V_lr, idxToImdbId = lr.process(df_movies, idxToLenId, links, features = ['year', 'votes', 'runtimes'])
U_lr = lr.train(X_tr, V_lr, lam = 1)

Below is the experiement result comparing with the collaberative filtering method as a baseline, using the some evaluation metric. Although the training error is higher, the testing error is actually lower than the collaberative filtering method. 

In [ ]:
# Evaluation
train_mse, test_mse = cf.error(X_tr, U_cf, V_cf), cf.error(X_te, U_cf, V_cf)
print "CF Train error: {:0.4f} Test error: {:0.4f}".format(train_mse, test_mse)

train_mse, test_mse = cf.error(X_tr, U_lr, V_lr), cf.error(X_te, U_lr, V_lr)
print "LR Train error: {:0.4f} Test error: {:0.4f}".format(train_mse, test_mse)

### Factorization Machine

### Same feature as linear regression

In [24]:
import math
from sklearn.feature_extraction import DictVectorizer
np.random.seed(10)
P = np.random.permutation(len(ratings))
n = len(ratings)
ratings_tr = ratings.iloc[P[:int(math.floor(9*n /10.))]]
ratings_te = ratings.iloc[P[int(math.floor(9*n /10.)):]]

In [57]:
import tokenizer
reload(tokenizer)

train_data, y_tr = tokenizer.tokenize(ratings_tr)
test_data, y_te = tokenizer.tokenize(ratings_te)

v = DictVectorizer()
X_train = v.fit_transform(train_data)
X_test = v.transform(test_data)

### Example (Rating matrix only)

In [63]:
def experiemnt(X_train, y_tr, X_test, y_te, iters = [1,3,5,10,25,50], num_factor = 3):
    models = []
    train_error, test_error = [], []

    for k in iters:
        model = pylibfm.FM(num_factors=num_factor, num_iter=k,
                        verbose=True, task="regression",
                        initial_learning_rate=1E-6, learning_rate_schedule="optimal")
        model.fit(X_train, y_tr)
        models.append(model)

        pred = model.predict(X_train)
        train_error.append(mean_squared_error(y_tr, preds))

        pred = model.predict(X_test)
        test_error.append(mean_squared_error(y_te, preds))

        return models, train_error, test_error


In [59]:
experiemnt(X_train, y_tr, X_test, y_te)

Creating validation dataset of 0.01 of training for adaptive regularization
-- Epoch 1
Training MSE: 5.19196
-- Epoch 2
Training MSE: 4.34369
-- Epoch 3
Training MSE: 3.20449
-- Epoch 4
Training MSE: 2.40837
-- Epoch 5
Training MSE: 1.85197
-- Epoch 6
Training MSE: 1.46310
-- Epoch 7
Training MSE: 1.19130
-- Epoch 8
Training MSE: 1.00132
-- Epoch 9
Training MSE: 0.86848
-- Epoch 10
Training MSE: 0.77559
-- Epoch 11
Training MSE: 0.71063
-- Epoch 12
Training MSE: 0.66516
-- Epoch 13
Training MSE: 0.63333
-- Epoch 14
Training MSE: 0.61102
-- Epoch 15
Training MSE: 0.59538
-- Epoch 16
Training MSE: 0.58438
-- Epoch 17
Training MSE: 0.57662
-- Epoch 18
Training MSE: 0.57115
-- Epoch 19
Training MSE: 0.56726
-- Epoch 20
Training MSE: 0.56448
-- Epoch 21
Training MSE: 0.56247
-- Epoch 22
Training MSE: 0.56101
-- Epoch 23
Training MSE: 0.55992
-- Epoch 24
Training MSE: 0.55910
-- Epoch 25
Training MSE: 0.55846
-- Epoch 26
Training MSE: 0.55795
-- Epoch 27
Training MSE: 0.55753
-- Epoch 28
Tra

In [60]:
preds = fm.predict(X_test)
from sklearn.metrics import mean_squared_error
print("FM MSE: %.4f" % mean_squared_error(y_te, preds))

FM MSE: 1.1066


In [ ]:
preds 

In [ ]:
df_movie_features_bkup

### References

F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages. DOI=http://dx.doi.org/10.1145/2827872